[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aldomunaretto/immune_deep_learning/blob/main/notebooks/01_intro_DL/03_keras_overfitting.ipynb)

# Artificial Neural Networks - Regularization

## Prevent Overfitting

### Import Libraries

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

from keras import Input, Model
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, BatchNormalization, GaussianNoise
from keras.utils import set_random_seed
from keras.regularizers import l1, l2, l1_l2
from keras.optimizers import SGD, Adam
from keras.callbacks import EarlyStopping, TensorBoard
from keras.datasets import mnist

import datetime
import numpy as np
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline

set_random_seed(0)

We will use the [MNIST dataset](https://en.wikipedia.org/wiki/MNIST_database), consisting of a collection of 28x28 pixel images corresponding in digits from 0 to 9 manuscripts. The purpose of this data set is to train models that recognize handwritten numbers.

We will train, therefore, an [OCR (Optical Character Recognition)](https://en.wikipedia.org/wiki/Optical_character_recognition) model for multiclass classification (numbers from 0 to 9).

In [ ]:
(x_train, y_train),(x_test, y_test) = mnist.load_data()

In [ ]:
x_train[0].shape

As they are 8-bit images, the colors (in this case only one channel, the gray channel) of each pixel are encoded with a value between 0 and 255, with 0 being black and 255 being white. It is usual to normalize the values to work with a range between 0.0 and 1.0.

In [ ]:
x_train, x_test = x_train / 255.0, x_test / 255.0

We visualize a random image of the training set.

In [ ]:
i = np.random.randint(0, len(x_train))
plt.imshow(x_train[i], cmap='gray')
plt.show()

We reduce the dataset for  inducing more overfitting

In [ ]:
# Split the dataset into training and testing sets
_, x, _, y = train_test_split(
    x_train, y_train, test_size=0.02, random_state=1, stratify=y_train)

In [ ]:
# Count the number of samples for each class
num_classes = 10
x.shape, y.shape, Counter(y)

#### Why do we need to use regularization?

We are going to assign the classes randomly to each image with random shuffle.

In [ ]:
y_shuffle = np.copy(y)
np.random.shuffle(y_shuffle)
y[:5], y_shuffle[:5]

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(9, 9))
axes = axes.ravel()

for i in np.arange(0, 9):
    ind = np.random.randint(len(y_shuffle))
    axes[i].imshow(x[ind].reshape(28, 28))
    axes[i].set_title(
        f"Digit = {y_shuffle[ind]}")
    axes[i].axis('off')

plt.subplots_adjust(wspace=0.3)

Lets build a model for learning the new random classes

In [ ]:
# Input
inputs = Input(shape=(28, 28))
# Convert the 2D image to a vector
flat = Flatten()(inputs)

# hidden layer 1
l_1 = Dense(64, activation='relu')(flat)
# hidden layer 2
l_2 = Dense(64, activation='relu')(l_1)
# hidden layer 3
l_3 = Dense(64, activation='relu')(l_2)

# Outputs
outputs = Dense(10, activation='softmax')(l_3)

# Model definition
model = Model(inputs=inputs, outputs=outputs)

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy']
              )

history = model.fit(
    x,
    y_shuffle,
    batch_size=64,
    epochs=100,
    shuffle=True
    )

In [ ]:
results = model.evaluate(x, y_shuffle, verbose=0)
print('Train Accuracy: {}'.format(results[1]))

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(9, 9))
axes = axes.ravel()

for i in np.arange(0, 9):
    ind = np.random.randint(len(y_shuffle))
    pred_pobs = model.predict(np.expand_dims(x[ind], 0)).flatten()
    pred_class = np.argmax(pred_pobs)
    prob = np.max(pred_pobs)
    axes[i].imshow(x[ind].reshape(28, 28))
    axes[i].set_title(
        f"Digit = {y_shuffle[ind]} \n Prediction={pred_class} , Prob={prob:.4f}")
    axes[i].axis('off')

plt.subplots_adjust(wspace=0.5)

We can see that the model has fully fitted the training set. It practically does not matter what data you introduce into the neural network, it will almost always fit. The objective is not to fit the training set, it is to **GENERALIZE**.

#### Regularization for preventing overfitting

To get the model to **generalize**, we can use one of the numerous regularization techniques

We build a model consisting of:

- Input with dimension (28,28)
- Flatten layer
- Dense hidden  layer with 256 neurons and ReLU as activation function
- Second dense hidden layer with 128 neurons and ReLU as activation function
- Third dense  hidden layer with 128 neurons and ReLU as activation function
- Output layer with a single neuron that implements the softmax function.

In [ ]:
# Input
inputs = Input(shape=(28,28), name='input_layer')
# Convert the 2D image to a vector
flat = Flatten()(inputs)

# hidden layer 1
l_1 = Dense(256, activation='relu', name='layer_1')(flat)
# hidden layer 2
l_2 = Dense(128, activation='relu', name='layer_2')(l_1)
# hidden layer 3
l_3 = Dense(128, activation='relu', name='layer_3')(l_2)

# Outputs
outputs = Dense(num_classes, activation='softmax', name='output_layer')(l_3)

# Model definition
model = Model(inputs=inputs, outputs=outputs, name='dont_overfit_model')

In [ ]:
# Model summary
model.summary()

We define the following configuration for training:
- **optimizer**: adam
- **loss function**: sparse_categorical_crossentropy
- **metrics**: accuracy

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy']
              )

In [ ]:
history = model.fit(
    x,
    y,
    batch_size=64,
    epochs=100,
    validation_split=0.2,
    shuffle=True
    )

In [ ]:
results = model.evaluate(x_test, y_test, verbose=0)
print('Test Loss: {}'.format(results[0]))
print('Test Accuracy: {}'.format(results[1]))

We visualize the evolution of the values in each epoch of the loss function and accuracy.

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch

In [ ]:
hist.head()

<a id='overfitting_results'></a>
When interpreting both graphs, it is evident that the model over-fits the training values and is not able to generalize well for the validation values. **It is a clear example of overfitting**.

The following function will be useful throughout the different practical examples.

In [ ]:
def show_loss_accuracy_evolution(history):

    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Categorical Crossentropy')
    ax1.plot(hist['epoch'], hist['loss'], label='Train Error')
    ax1.plot(hist['epoch'], hist['val_loss'], label = 'Val Error')
    ax1.grid()
    ax1.legend()

    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy')
    ax2.plot(hist['epoch'], hist['accuracy'], label='Train Accuracy')
    ax2.plot(hist['epoch'], hist['val_accuracy'], label = 'Val Accuracy')
    ax2.grid()
    ax2.legend()

    plt.show()

def show_samples(x_test, y_test, model):
    predictions = model.predict(x_test)
    predicted_classes = np.argmax(predictions, -1)

    L = 5
    W = 5
    fig, axes = plt.subplots(L, W, figsize=(14, 14))
    axes = axes.ravel()

    for i in np.arange(0, L * W):
        ind = np.random.randint(len(y_test))
        axes[i].imshow(x_test[ind].reshape(28, 28))
        prob_pred = np.max(predictions[ind, :])
        class_pred = int(predicted_classes[ind])
        original_class = int(y_test[ind])
        if class_pred == original_class:
            color = 'green'
        else:
            color = 'red'
        title = "Pred: {0} \n Target: {1} \n Prob: {2:.3f}".format(
        class_pred, original_class, prob_pred)
        axes[i].set_title(title, color=color)
        axes[i].axis('off')

    plt.subplots_adjust(wspace=0.5)

In [ ]:
show_loss_accuracy_evolution(history)

In [ ]:
results = model.evaluate(x_test, y_test, verbose=0)
print('Test Loss: {}'.format(results[0]))
print('Test Accuracy: {}'.format(results[1]))

In [ ]:
show_samples(x_test, y_test, model)

## Create a simpler model

The simplest way to prevent overfitting is to create  a simple model: A model with less number of  parameters (which is determined by the number of layers and the number of units per layer).

A more complex model with more parameters will have much more capacity and will be able to learn almost any training set. Deep learning models tend to be good at fitting to the training data, but the real challenge is generalization, not fitting.

In [ ]:
# Input layer
inputs = Input(shape=(28, 28), name='input_layer')
flat = Flatten()(inputs)

# First hidden layer
l_1 = Dense(8, activation='relu', name='layer_1')(flat)

# Output layer
outputs = Dense(num_classes, activation='softmax', name='output_layer')(l_1)

# Model definition
model_simpler = Model(inputs=inputs, outputs=outputs, name='simpler_model')

In [ ]:
## Model summary
model_simpler.summary()

In [ ]:
model_simpler.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
history_simpler = model_simpler.fit(x,
                                    y,
                                    batch_size=64,
                                    epochs=100,
                                    validation_split=0.25,
                                    shuffle=True)

In [ ]:
show_loss_accuracy_evolution(history_simpler)

In [ ]:
results = model_simpler.evaluate(x_test, y_test, verbose=0)
print('Test Loss: {}'.format(results[0]))
print('Test Accuracy: {}'.format(results[1]))

## Reduce batch_size

Other method is decreasing the `batch_size`during the gradient descend to add more uncertainty to the estimation of the parameter gradients, thus reducing the capacity of the model

In [ ]:
# Input
inputs = Input(shape=(28,28), name='input_layer')
# Convert the 2D image to a vector
flat = Flatten()(inputs)

# hidden layer 1
l_1 = Dense(256, activation='relu', name='layer_1')(flat)
# hidden layer 2
l_2 = Dense(128, activation='relu', name='layer_2')(l_1)
# hidden layer 3
l_3 = Dense(128, activation='relu', name='layer_3')(l_2)

# Outputs
outputs = Dense(num_classes, activation='softmax', name='output_layer')(l_3)

# Model definition
model = Model(inputs=inputs, outputs=outputs, name='dont_overfit_model')

In [ ]:
## Model summary
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
history_2 = model.fit(
    x,
    y,
    batch_size=16,
    epochs=100,
    validation_split=0.2,
    shuffle=True
)

In [ ]:
show_loss_accuracy_evolution(history_2)

In [ ]:
results = model.evaluate(x_test, y_test, verbose=0)
print('Test Loss: {}'.format(results[0]))
print('Test Accuracy: {}'.format(results[1]))

### Practice 1:
- What differences do you see if you change the batch_size to a lower or higher value? For example to 1 and to 512

<a id='dropout'></a>
## Dropout

Dropout, applied to a layer, consists of randomly "dropping out" (set to zero) a number of output features of the layer during training with a probability **p** (`dropout rate`).

In `keras` you can introduce dropout in a network via the [Dropout layer](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout), which gets applied to the output of layer right before.

```python
keras.layers.Dropout(
    rate, noise_shape=None, seed=None, **kwargs
)
```

With Functional API:
```python
next_layer = Dropout(0.4)(prev_layer)
```
With Sequential:
```python
model.add(Dropout(0.4))
```

In [ ]:
# Input
inputs = Input(shape=(28, 28), name='input_layer')
# Convert the 2D image to a vector
flat = Flatten()(inputs)

flat = Dropout(0.5, name='dropout_flat')(flat)
# hidden layer 1
l_1 = Dense(256, activation='relu', name='layer_1')(flat)
l_1 = Dropout(0.5, name='dropout_l1')(l_1)
# hidden layer 2
l_2 = Dense(128, activation='relu', name='layer_2')(l_1)
l_2 = Dropout(0.5, name='dropout_l2')(l_2)
# hidden layer 3
l_3 = Dense(128, activation='relu', name='layer_3')(l_2)
l_3 = Dropout(0.5, name='dropout_l3')(l_3)

# Outputs
outputs = Dense(num_classes, activation='softmax', name='output_layer')(l_3)

# Model definition
model_dropout = Model(inputs=inputs, outputs=outputs, name='dont_overfit_model_dropout')

In [ ]:
# Model summary
model_dropout.summary()

In [ ]:
model_dropout.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])

In [ ]:
history_dropout = model_dropout.fit(
    x,
    y,
    batch_size=64,
    epochs=100,
    validation_split=0.2,
    shuffle=True
)

In [ ]:
show_loss_accuracy_evolution(history_dropout)

In [ ]:
results = model_dropout.evaluate(x_test, y_test, verbose=0)
print('Test Loss: {}'.format(results[0]))
print('Test Accuracy: {}'.format(results[1]))

### Practice 2:
- What happens if you change the dropout to 0.1 or 0.9? 
- And what if you put it only on one layer?
- What happens if we delete the flatten first dropout?

## Batch Normalization

[Batch normalization](https://www.tensorflow.org/api_docs/python/tf/keras/layers/BatchNormalization) applies a transformation that maintains the mean output close to 0 and the output standard deviation close to 1.


With Functional API:
```python
next_layer = layers.BatchNormalization()(prev_layer)
```
With Sequential:
```python
model.add(layers.BatchNormalization())
```

In [ ]:
# Input
inputs = Input(shape=(28, 28), name='input_layer')
# Convert the 2D image to a vector
flat = Flatten()(inputs)

# hidden layer 1
l_1 = Dense(256, activation='relu', name='layer_1')(flat)
l_1 = BatchNormalization()(l_1)
# hidden layer 2
l_2 = Dense(128, activation='relu', name='layer_2')(l_1)
l_2 = BatchNormalization()(l_2)
# hidden layer 3
l_3 = Dense(128, activation='relu', name='layer_3')(l_2)

# Outputs
outputs = Dense(num_classes, activation='softmax', name='output_layer')(l_3)

# Model definition
model_batch_norm = Model(inputs=inputs, outputs=outputs, name='dont_overfit_model_batch_norm')

In [ ]:
# Model summary
model_batch_norm.summary()

In [ ]:
model_batch_norm.compile(optimizer='adam', 
                         loss='sparse_categorical_crossentropy', 
                         metrics=['accuracy'])

In [ ]:
history_batch_norm = model_batch_norm.fit(
    x,
    y,
    batch_size=64,
    epochs=100,
    validation_split=0.2,
    shuffle=True
)

In [ ]:
show_loss_accuracy_evolution(history_batch_norm)

In [ ]:
results = model_batch_norm.evaluate(x_test, y_test, verbose=0)
print('Test Loss: {}'.format(results[0]))
print('Test Accuracy: {}'.format(results[1]))

<a id='l1_l2_elasticnet'></a>
## L1, L2 and ElasticNet

Remember that during training the configuration of weights and biases is learned to improve the results for a given loss function.

* [L1 regularization](https://developers.google.com/machine-learning/glossary/#L1_regularization), where the cost added is proportional to the absolute value of the weights coefficients ("L1 norm" of the weights).

```python
keras.regularizers.l1(l1=0.01)

Dense(3, kernel_regularizer='l1')

```

* [L2 regularization](https://developers.google.com/machine-learning/glossary/#L2_regularization), where the cost added is proportional to the square of the value of the weights coefficients ( "L2 norm" of the weights).

```python
keras.regularizers.l2(l2=0.01)

Dense(3, kernel_regularizer='l2')

```
* L1_L2 or Elastic Net.

```python
keras.regularizers.l1_l2(
    l1=0.01, l2=0.01
)

Dense(3, kernel_regularizer='l1_l2')

```

Furthermore, it is possible to choose whether to include the penalty in the cost function on the weights, the biases or on the activation, with the following arguments:
- `kernel_regularizer`: only on weights.
- `bias_regularizer`: only on biases.
- `activity_regularizer`: on full output.

[link to documentation](https://keras.io/api/layers/regularizers/)

In [ ]:
kerner_regularizer_l1 = l1_l2(l1=1e-5, l2=5e-4)
kerner_regularizer_l2 = l2(5e-4)
kerner_regularizer_l3 = l1(1e-5)

In [ ]:

# Input
inputs = Input(shape=(28, 28), name='input_layer')
# Convert the 2D image to a vector
flat = Flatten()(inputs)

# hidden layer 1
l_1 = Dense(256, activation='relu', kernel_regularizer=kerner_regularizer_l1, name='layer_1')(flat)
# hidden layer 2
l_2 = Dense(128, activation='relu', kernel_regularizer=kerner_regularizer_l2, name='layer_2')(l_1)
# hidden layer 3
l_3 = Dense(128, activation='relu', kernel_regularizer=kerner_regularizer_l3, name='layer_3')(l_2)

# Outputs
outputs = Dense(num_classes, activation='softmax', name='output_layer')(l_3)

# Model definition
model_regularizers = Model(inputs=inputs, outputs=outputs, name='dont_overfit_model_regularizers')

In [ ]:
# Model summary
model_regularizers.summary()

In [ ]:
model_regularizers.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history_regularizers = model_regularizers.fit(
    x,
    y,
    batch_size=64,
    epochs=100,
    validation_split=0.2,
    shuffle=True
)

Lets compare the weights norms of the model without regularization and with regularization.

In [ ]:
for ly in model.layers:
    if 'layer_' in ly.name:
        W, b = ly.get_weights()
        l2_norm = np.sum(W**2)**0.5
        zero_elems = len(W[np.abs(W) < 1e-14])
        sparsity = zero_elems / np.prod(W.shape)
        sparsity = np.round(100 * sparsity, 2)
        print('L2 norm {0} weights: {1}, shape:{2}, sparsity:{3}'.format(
            ly.name, l2_norm, W.shape,sparsity))

In [ ]:
for ly in model_regularizers.layers:
    if 'layer_' in ly.name:
        W, b = ly.get_weights()
        l2_norm = np.sum(W**2)**0.5
        zero_elems = len(W[np.abs(W) < 1e-14])
        sparsity = zero_elems / np.prod(W.shape)
        sparsity = np.round(100 * sparsity, 2)
        print('L2 norm {0} weights: {1}, shape:{2}, sparsity:{3}'.format(
            ly.name, l2_norm, W.shape,sparsity))

In [ ]:
for ly in model_batch_norm.layers:
    if 'layer_' in ly.name:
        W, b = ly.get_weights()
        l2_norm = np.sum(W**2)**0.5
        zero_elems = len(W[np.abs(W) < 1e-14])
        sparsity = zero_elems / np.prod(W.shape)
        sparsity = np.round(100 * sparsity, 2)
        print('L2 norm {0} weights: {1}, shape:{2}, sparsity:{3}'.format(
            ly.name, l2_norm, W.shape,sparsity))

In [ ]:
show_loss_accuracy_evolution(history_regularizers)

In [ ]:
results = model_regularizers.evaluate(x_test, y_test, verbose=0)
print('Test Loss: {}'.format(results[0]))
print('Test Accuracy: {}'.format(results[1]))

### Practice 3:
- What happens if you change the regularizers?


#### Combine L2 Regularization and Dropout

Use L2:
```python
kerner_regularizer_l2 = l2(5e-4)
```
And Dropout:

```python
next_layer = Dropout(0.4)(prev_layer)
```

In [ ]:
kerner_regularizer_l2 = ...

In [ ]:
inputs = Input(shape=(), name='input_layer')

# Convert the 2D image to a vector
flat = (inputs)


# Add L2-normalization
l_1 = Dense(128, activation='relu',
                   kernel_regularizer=...,
                   name='layer_1')(flat)

# Add dropout
l_1 = ...(l_1)


# Add L2-normalization
l_2 = Dense(64,
                   activation='relu',
                   kernel_regularizer=...,
                   name='layer_2')(l_1)
# Add dropout
l_2 = ...(l_2)


# Add L2-normalization
l_3 = Dense(32,
                   activation='relu',
                   kernel_regularizer=...,
                   name='layer_3')(l_2)

# Add dropout
l_3 = ...(l_3)

# Output layer
outputs = Dense(..., activation=...,
                       name='output_layer')(l_3)

# Model definition
model_combination = Model(inputs=inputs, outputs=outputs, name='dont_overfit_model_regularizers')

In [ ]:
model_combination.compile(optimizer='adam', 
                          loss='sparse_categorical_crossentropy', 
                          metrics=['accuracy'])

In [ ]:
history_combination = model_combination.fit(
    x,
    y,
    batch_size=64,
    epochs=100,
    validation_split=0.2,
    shuffle=True
)

In [ ]:
show_loss_accuracy_evolution(history_combination)

In [ ]:
results = model_combination.evaluate(x_test, y_test, verbose=0)
print('Test Loss: {}'.format(results[0]))
print('Test Accuracy: {}'.format(results[1]))

## Noise injection

Apply additive zero-mean Gaussian noise.

With Functional API:
```python
next_layer = GaussianNoise(stddev)(prev_layer)
```
With Sequential:
```python
model.add(GaussianNoise(stddev)())
```

In [ ]:
stddev = 0.2

# Input
inputs = Input(shape=(28,28), name='input_layer')
# Convert the 2D image to a vector
flat = Flatten()(inputs)

# hidden layer 1
l_1 = Dense(256, activation='relu', name='layer_1')(flat)
l_1 = GaussianNoise(stddev, name='noise_l1')(l_1)
# hidden layer 2
l_2 = Dense(128, activation='relu', name='layer_2')(l_1)
l_2 = GaussianNoise(stddev, name='noise_l2')(l_2)
# hidden layer 3
l_3 = Dense(128, activation='relu', name='layer_3')(l_2)
l_3 = GaussianNoise(stddev, name='noise_l3')(l_3)

# Outputs
outputs = Dense(num_classes, activation='softmax', name='output_layer')(l_3)

# Model definition
model_noise = Model(inputs=inputs, outputs=outputs, name='dont_overfit_model_noise')

In [ ]:
# Model summary
model_noise.summary()

In [ ]:
model_noise.compile(optimizer='adam', 
                    loss='sparse_categorical_crossentropy', 
                    metrics=['accuracy'])

In [ ]:
history_noise = model_noise.fit(
    x,
    y,
    batch_size=64,
    epochs=100,
    validation_split=0.2,
    shuffle=True
)

In [ ]:
show_loss_accuracy_evolution(history_noise)

In [ ]:
results = model_noise.evaluate(x_test, y_test, verbose=0)
print('Test Loss: {}'.format(results[0]))
print('Test Accuracy: {}'.format(results[1]))

<a id='early_stopping'></a>
## Early Stopping

Another of the most used techniques in neural network training is **early stopping**. It basically consists of stopping the training process in an Epoch prior to the one defined in the compilation.

This, in addition to resulting in a model with less overfitting, saves unnecessary computation time.

To do this, we can use the [Keras callbacks](https://keras.io/api/callbacks/), objects that perform actions at different times during training. Specifically, we will make use of [`EarlyStopping`](https://keras.io/api/callbacks/early_stopping/).

```python
keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=0,
    restore_best_weights=False,
    verbose=0,
)
```

- **monitor**: Quantity to be monitored. Depending on the evolution, it will be decided to stop training.

- **min_delta**: Minimum change in the monitored quantity to qualify as an improvement, i.e. an absolute change of less than min_delta, will count as no improvement.

- **patience**: Number of epochs with no improvement after which training will be stopped.

- **restor_best_weights**: Whether to restore model weights from the epoch with the best value of the monitored quantity

- **verbose**: verbosity mode.

There are other callbacks that can be very useful such as:
- [`ModelCheckpoint`](https://keras.io/api/callbacks/model_checkpoint/), which allows storing the state of a model at different times of training. This is very useful in trainings that can take hours or days.
- [`TensorBoard`](https://keras.io/api/callbacks/tensorboard/), which allows the use of [TensorBoard](https://www.tensorflow.org/tensorboard?hl=es-419), framework TensorFlow for visualizing metrics and evaluating models.

In [ ]:
es_callback = EarlyStopping(
    monitor='val_loss',  # can be 'val_accuracy'
    patience=5,  # if during 5 epochs there is no improvement in `val_loss`, the execution will stop
    restore_best_weights=True,
    verbose=1)

Lets create the same model:

In [ ]:
# Input
inputs = Input(shape=(28,28), name='input_layer')
# Convert the 2D image to a vector
flat = Flatten()(inputs)

# hidden layer 1
l_1 = Dense(256, activation='relu', name='layer_1')(flat)
# hidden layer 2
l_2 = Dense(128, activation='relu', name='layer_2')(l_1)
# hidden layer 3
l_3 = Dense(128, activation='relu', name='layer_3')(l_2)

# Outputs
outputs = Dense(num_classes, activation='softmax', name='output_layer')(l_3)

# Model definition
model_early_stopping = Model(inputs=inputs, outputs=outputs, name='dont_overfit_model_early_stopping')

In [ ]:
# Model summary
model_early_stopping.summary()

In [ ]:
model_early_stopping.compile(optimizer='adam',
                             loss='sparse_categorical_crossentropy',
                             metrics=['accuracy'])

In [ ]:
history_early_stopping = model_early_stopping.fit(
    x,
    y,
    batch_size=64,
    epochs=100,
    validation_split=0.2,
    shuffle=True,
    callbacks=[es_callback]
)

In [ ]:
show_loss_accuracy_evolution(history_early_stopping)

In [ ]:
results = model_early_stopping.evaluate(x_test, y_test, verbose=0)
print('Test Loss: {}'.format(results[0]))
print('Test Accuracy: {}'.format(results[1]))

<a id='learning_rate'></a>
## Adjusting the learning rate

By adjusting the learning rate we can obtain better results and avoid falling into local minimums.

```python
keras.optimizers.Adam(learning_rate=0.001)
```

We can use alsa `SGD` optimizer or any other optimizer.

In [ ]:
# Input
inputs = Input(shape=(28,28), name='input_layer')
# Convert the 2D image to a vector
flat = Flatten()(inputs)

# hidden layer 1
l_1 = Dense(256, activation='relu', name='layer_1')(flat)
# hidden layer 2
l_2 = Dense(128, activation='relu', name='layer_2')(l_1)
# hidden layer 3
l_3 = Dense(128, activation='relu', name='layer_3')(l_2)

# Outputs
outputs = Dense(num_classes, activation='softmax', name='output_layer')(l_3)

# Model definition
model_learning_rate = Model(inputs=inputs, outputs=outputs, name='dont_overfit_model_learning_rate')

In [ ]:
# Model summary
model_learning_rate.summary()

In [ ]:
model_learning_rate.compile(optimizer=Adam(learning_rate=0.0003),
                             loss='sparse_categorical_crossentropy',
                             metrics=['accuracy'])

In [ ]:
history_learning_rate = model_learning_rate.fit(
    x,
    y,
    batch_size=64,
    epochs=100,
    validation_split=0.2,
    shuffle=True,
)

In [ ]:
show_loss_accuracy_evolution(history_learning_rate)

In [ ]:
results = model_learning_rate.evaluate(x_test, y_test, verbose=0)
print('Test Loss: {}'.format(results[0]))
print('Test Accuracy: {}'.format(results[1]))

<a id='weight_decay'></a>
## Weight decay

By adding a penalty to the loss function through weight decay, we allow the network to be optimized while its weights do not increase too much.

```python
keras.optimizers.Adam(weight_decay=None)
```

We can use alsa `SGD` optimizer or any other optimizer.


In [ ]:
# Input
inputs = Input(shape=(28,28), name='input_layer')
# Convert the 2D image to a vector
flat = Flatten()(inputs)

# hidden layer 1
l_1 = Dense(256, activation='relu', name='layer_1')(flat)
# hidden layer 2
l_2 = Dense(128, activation='relu', name='layer_2')(l_1)
# hidden layer 3
l_3 = Dense(128, activation='relu', name='layer_3')(l_2)

# Outputs
outputs = Dense(num_classes, activation='softmax', name='output_layer')(l_3)

# Model definition
model_weight_decay = Model(inputs=inputs, outputs=outputs, name='dont_overfit_model_weight_decay')

In [ ]:
# Model summary
model_weight_decay.summary()

In [ ]:
model_weight_decay.compile(optimizer=Adam(weight_decay=1e-4),
                           loss='sparse_categorical_crossentropy',
                           metrics=['accuracy'])

In [ ]:
history_weight_decay = model_weight_decay.fit(
    x,
    y,
    batch_size=64,
    epochs=100,
    validation_split=0.2,
    shuffle=True,
)

In [ ]:
show_loss_accuracy_evolution(history_weight_decay)

In [ ]:
results = model_weight_decay.evaluate(x_test, y_test, verbose=0)
print('Test Loss: {}'.format(results[0]))
print('Test Accuracy: {}'.format(results[1]))

<a id='tensorboard'></a>
# TensorBoard

Load TensorBoard notebook extension.

In [ ]:
%load_ext tensorboard


We build the model. The input layer will have a dimension of `28x28`. The output layer, in a multiclass classification problem with 10 possible classes, will be made up of 10 neurons with [softmax activation function](https://keras.io/api/layers/activations/#softmax-function ).


In [ ]:
kernel_regularizer_l2 = l2(5e-4)

model = Sequential(name="tensorboard_model")
model.add(Input(shape=(28, 28)))
model.add(Flatten())

# Hidden Layers + regularization
model.add(Dense(64, activation='relu', name='layer_1'))
model.add(Dense(64, activation='relu',kernel_regularizer=kernel_regularizer_l2, name='layer_2'))
# output layer
model.add(Dense(10, activation='softmax', name='output_layer'))

In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In the [example with early stopping](#early_stopping) the Keras callbacks were presented. To use TensorBoard we will need to include the [`TensorBoard` callback](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/TensorBoard) to the training.

This callback will create and store the logs in the directory that we indicate.

By default it will create a directory called "logs" in the directory where this notebook is running.

In [ ]:
log_dir = 'logs/fit/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')

In [ ]:
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
es_callback = EarlyStopping(
    monitor='val_loss',  # can be 'val_accuracy'
    patience=10,  # if during 10 epochs there is no improvement in `val_loss`, the execution will stop
    restore_best_weights=True,
    verbose=1)

Train the model

In [ ]:
model.fit(
    x=x_train,
    y=y_train,
    epochs=20,
    validation_split=0.2,
    batch_size=32,
    callbacks=[tensorboard_callback, es_callback]
)

In [ ]:
results = model.evaluate(x_test, y_test, verbose=0)
print('Test Loss: {}'.format(results[0]))
print('Test Accuracy: {}'.format(results[1]))

To ** start TensorBoard ** we can execute the following cell or, from the command line, execute:

    tensorboard --logdir logs / fit

An application will be launched that will be listening by default on port 6006. It is possible to use TensorBoard by opening [localhost: 6006 /](http://localhost:6006/) in a browser.

In [ ]:
%tensorboard --logdir logs/fit

A brief overview of the dashboards shown (tabs in top navigation bar):

* The **Scalars** dashboard shows how the loss and metrics change with every epoch. You can use it to also track training speed, learning rate, and other scalar values.
* The **Graphs** dashboard helps you visualize your model. In this case, the Keras graph of layers is shown which can help you ensure it is built correctly.
* The **Distributions** and **Histograms** dashboards show the distribution of a Tensor over time. This can be useful to visualize weights and biases and verify that they are changing in an expected way.



### Practice 4 
- Improve the last model with some regularization techniques to obtain at least `0.98` of test accuracy

In [ ]:
model = Sequential()
model.add(Input(shape=(28, 28)))
model.add(Flatten())

# Hidden Layers + regularization
model.add(...)
# Output layer
model.add(Dense(10, activation='softmax'))

In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model.fit(
    x=x_train,
    y=y_train,
    epochs=5,
    validation_split=0.2,
    batch_size=64
)
results = model.evaluate(x_test, y_test, verbose=0)
print('Test Loss: {}'.format(results[0]))
print('Test Accuracy: {}'.format(results[1]))

In [ ]:
model.fit(
    x=x_train,
    y=y_train,
    epochs=...,
    validation_split=0.2,
    batch_size=...,
    callbacks=...
)
results = model.evaluate(x_test, y_test, verbose=0)
print('Test Loss: {}'.format(results[0]))
print('Test Accuracy: {}'.format(results[1]))

### Plotting Predictions

In [ ]:
predictions = model.predict(x_test)
predicted_classes = np.argmax(predictions, -1)

In [ ]:
L = 5
W = 5
fig, axes = plt.subplots(L, W, figsize=(14, 14))
axes = axes.ravel()

for i in np.arange(0, L * W):
    ind = np.random.randint(len(y_test))
    axes[i].imshow(x_test[ind].reshape(28, 28))
    prob_pred = np.max(predictions[ind, :])
    class_pred = int(predicted_classes[ind])
    original_class = int(y_test[ind])
    if class_pred == original_class:
        color = 'green'
    else:
        color = 'red'
    title = "Pred: {0} \n Target: {1} \n Prob: {2:.3f}".format(
    class_pred, original_class, prob_pred)
    axes[i].set_title(title, color=color)
    axes[i].axis('off')

plt.subplots_adjust(wspace=0.5)